<a href="https://colab.research.google.com/github/b21renu/ESSENCE/blob/main/Task2ESSENCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
T5, GPT, Bert, XLNet, TinyBert
2) Fine-tune the models on the dataset and see what is the results? Let’s take the "task type" classes. Mind that your train-val split
should include all classes, so something like StratifiedSplit from sklearn would help.

In our dataset (https://drive.google.com/file/d/1cwhVDiBWDu5xUwYDLfVTClpqzETcJaEF/view?usp=sharing) there are also ChatGPT-generated tasks.
I think you could drop them from the dataset for the start.
```

```
StratifiedShuffleSplit from scikit-learn: This ensures that the distribution of classes in both the training and validation sets
remains similar to that of the original dataset.
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch
import time
import random

In [ ]:
path = '/content/drive/MyDrive/RVU S4/INTERNSHIP/dataset-20240111.csv'
df = pd.read_csv(path)
df.head()

,Task name,Type,Intensity,Generated
0,spa,Rest & self-care,Low / Medium,0.0
1,doctor,Rest & self-care,Low / Medium,0.0
2,read,Rest & self-care,Low / Medium,0.0
3,therapist,Rest & self-care,High,0.0
4,meditate,Rest & self-care,High,0.0


In [ ]:
del df['Generated']

In [ ]:
df.isnull().any()

Task name    False
Type         False
Intensity    False
dtype: bool

# GPT

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Generate text based on a random sentence from the DataFrame
random_index = random.randint(0, len(df) - 1)
random_sentence = df.loc[random_index, 'Task name']
print(f"Randomly selected sentence: {random_index} - {random_sentence}")

# Measure tokenization time
start_time = time.time()

# Tokenize and encode the input text
input_ids = tokenizer.encode(random_sentence, return_tensors="pt")

end_time = time.time()

# Generate text based on the tokenized input
output = model.generate(input_ids, max_length=120, temperature=0.8, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

# Calculate tokenization time
tokenization_time = end_time - start_time
print(f"Tokenization Time: {tokenization_time} seconds")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Randomly selected sentence: 249 - Plan CSR activity for next quarter
Plan CSR activity for next quarter is expected to be $1.3 billion, up from $2.6 billion in the same period last year.

"This is a significant increase from the $3.2 billion we saw in Q1, and we expect this to continue to grow over the next few quarters," he said in a statement.
Tokenization Time: 0.0010211467742919922 seconds


In [ ]:
# Initialize an empty list to store tokenization times
tokenization_times = []

# Tokenize each sentence in the DataFrame
for sentence in df['Task name']:
    # Measure tokenization time
    start_time = time.time()

    # Tokenize and encode the input text
    input_ids = tokenizer.encode(sentence, return_tensors="pt")

    end_time = time.time()

    # Calculate tokenization time and append to the list
    tokenization_time = end_time - start_time
    tokenization_times.append(tokenization_time)

# Calculate average tokenization time
average_tokenization_time = sum(tokenization_times) / len(tokenization_times)
print(f"Average Tokenization Time: {average_tokenization_time} seconds")


Average Tokenization Time: 0.00021738546520638067 seconds


In [ ]:
import random
import time
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from transformers import AdamW, GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
# # Prepare the Data
# X = df[['Task name', 'Intensity']].apply(lambda x: ' '.join(x), axis=1).tolist()
# y = df['Type'].tolist()

In [ ]:
# Prepare the Data
X_train = df.loc[y_train.index, ['Task name', 'Intensity']].apply(lambda x: ' '.join(x), axis=1).tolist()
X_val = df.loc[y_val.index, ['Task name', 'Intensity']].apply(lambda x: ' '.join(x), axis=1).tolist()
y_train = df.loc[y_train.index, 'Type'].tolist()
y_val = df.loc[y_val.index, 'Type'].tolist()


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Split the dataset into train and validation sets using stratified split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Define training loop
def train_model(model, tokenizer, train_dataset, val_dataset, epochs=3, batch_size=8, learning_rate=2e-5):
    # Define optimizer and loss function
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    loss_fn = torch.nn.CrossEntropyLoss()

    # DataLoaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_batch)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        # Training loop
        for batch in train_loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=batch, labels=batch)
            loss = outputs.loss

            # Backward pass
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation loop
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                outputs = model(input_ids=batch, labels=batch)
                val_loss += outputs.loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

In [ ]:
# Function to pad sequences within a batch
def collate_batch(batch):
    # Tokenize and encode each sentence in the batch
    tokenized_batch = [tokenizer(item, return_tensors="pt", padding='max_length', truncation=True, max_length=512) for item in batch]

    # Extract input_ids from tokenized batch
    input_ids_batch = [item["input_ids"].squeeze(0) for item in tokenized_batch]

    # Pad sequences within the batch to the same length
    padded_input_ids_batch = pad_sequence(input_ids_batch, batch_first=True, padding_value=tokenizer.pad_token_id)

    return padded_input_ids_batch

In [ ]:
# Fine-tune the model
train_model(model, tokenizer, train_dataset, val_dataset, epochs=3)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import time
import random

In [ ]:
# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Prepare the Data
X = df[['Task name', 'Intensity']].apply(lambda x: ' '.join(x), axis=1).tolist()
y = df['Type'].tolist()

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 3: Define the Dataset
class CustomDataset(Dataset):
    def __init__(self, tokenizer, texts):
        self.tokenizer = tokenizer
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, return_tensors="pt")
        return inputs

train_dataset = CustomDataset(tokenizer, X_train)
val_dataset = CustomDataset(tokenizer, X_val)

In [ ]:
# Step 4: Modify the Model
# No modification needed for GPT-2 LMHeadModel for text generation

# Step 5: Specify Hyperparameters
batch_size = 4
num_epochs = 3
learning_rate = 5e-5

In [ ]:
# from torch.nn.utils.rnn import pad_sequence

# # Add special tokens to the tokenizer (including padding token)
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # You can choose any string for padding

# # Custom collate function to pad sequences within each batch
# def collate_fn(batch):
#     input_ids = [item['input_ids'] for item in batch]
#     padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
#     return {'input_ids': padded_input_ids}

In [ ]:
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    max_len = max(len(ids) for ids in input_ids)
    print("Batch max length:", max_len)
    padded_input_ids = [torch.cat((ids, torch.tensor([tokenizer.pad_token_id] * (max_len - len(ids)), dtype=torch.long))) for ids in input_ids]
    return {'input_ids': pad_sequence(padded_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)}


In [ ]:
# Step 6: Fine-tune the Model
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        outputs = model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

Batch max length: 1


RuntimeError: The size of tensor a (7) must match the size of tensor b (11) at non-singleton dimension 1

In [ ]:
# Step 7: Evaluate Performance (optional for text generation)

# Generate text based on the tokenized input
def generate_text(text):
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
    output = model.generate(input_ids, max_length=120, temperature=0.8, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output

In [ ]:
# Example of generating text
random_index = random.randint(0, len(X_val) - 1)
random_sentence = X_val[random_index]
generated_text = generate_text(random_sentence)
print("Random Sentence:", random_sentence)
print("Generated Text:", generated_text)

# TinyBERT

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

In [ ]:
import random
import time
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# Prepare the Data
X = df[['Task name', 'Intensity']].apply(lambda x: ' '.join(x), axis=1).tolist()
y = df['Type'].tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Assuming binary classification


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate>=0.21.0

In [ ]:
!pip install accelerate -U

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./output',                  # Directory for storing the trained model
    per_device_train_batch_size=4,          # Batch size per GPU
    num_train_epochs=3,                     # Number of training epochs
    logging_dir='./logs',                   # Directory for storing logs
    logging_steps=500,                      # Log every n steps
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=list(zip(X_train, y_train)),
    eval_dataset=list(zip(X_val, y_val)),
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()